### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.write("color_string =\n")
  output_file.write("visual_inspection_comments =\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [6]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [7]:
num_iterations = 1
empty_list = []
for i in range(7900, 8900):

    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.4548062975818388
cusum = 0.23762387521930262
eta = 1.473644587158249
hl_amp_ratio = 0.3350179341498844
kurtosis = -0.5307528230975778
n_points = 974
period = 0.5037006497921159
period_SNR = 198.37451780002698
period_log10FAP = -134.99819182204251
period_uncertainty = 9.027540779366072e-05
phase_cusum = 0.37894619796848283
phase_eta = 0.2408169679303064
phi21 = 0.5931651759114391
phi31 = 1.4754682304125177
quartile31 = 0.6215229026215852
r21 = 0.4228961811848787
r31 = 0.3072465369852081
shapiro_w = 0.9077775478363037
skewness = -0.7516987663591638
slope_per10 = -0.030460452692849017
slope_per90 = 0.031037555188442187
stetson_k = 0.6681512066591175
weighted_mean = 14.748456272836215
weighted_std = 0.399836855552505
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.13878288539006425
cusum = 0.20774957572136646
eta = 1.5062088177312198
hl_amp_ratio = 0.7381134873030264
kurtosis = -0.54033161046477
n_points = 982
period = 0.49847634952918496
period_SNR = 111.16466892227736
period_log10FAP = -61.12456151778001
period_uncertainty = 0.00011087930025791004
phase_cusum = 0.20470711872255548
phase_eta = 1.0824923883973714
phi21 = 0.3396471229493712
phi31 = 0.8131048261581554
quartile31 = 0.2578863924104855
r21 = 0.5200545902131358
r31 = 0.34762522430612525
shapiro_w = 0.9840418696403503
skewness = -0.27409326766790326
slope_per10 = -0.01639142374293013
slope_per90 = 0.019426494210527716
stetson_k = 0.7312666252784114
weighted_mean = 14.621996119810612
weighted_std = 0.18322751608989002
Pulled 1 of 1


...



######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.93


######### Lightcurve features #########

amplitude = 0.1495079771438131
cusum = 0.11697917512960397
eta = 2.2050435990801143
hl_amp_ratio = 0.6442582507771085
kurtosis = -0.3033300994180421
n_points = 1371
period = 0.6998647229557392
period_SNR = 216.83677684167813
period_log10FAP = -152.57907769718915
period_uncertainty = 0.00017340554118083773
phase_cusum = 0.2672734950824951
phase_eta = 0.9702191612212296
phi21 = 3.556811654335806
phi31 = 4.76030070185683
quartile31 = 0.21391079467657548
r21 = 0.27036162817085857
r31 = 0.13273426349233056
shapiro_w = 0.9888862371444702
skewness = -0.2544959384730102
slope_per10 = -0.029497605045549585
slope_per90 = 0.0342494167574704
stetson_k = 0.7550894268220626
weighted_mean = 15.454884131670521
weighted_std = 0.1351983777839589
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_e
class_probability = 0.7


######### Lightcurve features #########

amplitude = 0.1339975074112895
cusum = 0.12574137232392119
eta = 1.692977372018951
hl_amp_ratio = 0.9381618793818561
kurtosis = -0.7116110364448516
n_points = 904
period = 0.2517807252036395
period_SNR = 194.6556680703491
period_log10FAP = -129.12814353459893
period_uncertainty = 2.2687036147606143e-05
phase_cusum = 0.3083382886252689
phase_eta = 0.7596546061006955
phi21 = 3.9442045668024615
phi31 = 3.1486515340522203
quartile31 = 0.1837850323656074
r21 = 0.055976931170242994
r31 = 0.028350342341495836
shapiro_w = 0.988387405872345
skewness = -0.009119412475984433
slope_per10 = -0.028308702965283743
slope_per90 = 0.021764792273280098
stetson_k = 0.815910030602744
weighted_mean = 15.30535874293836
weighted_std = 0.10663181401277382
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


KeyboardInterrupt: 